In [ ]:
# Useful imports
import csv as csv
import pandas as pd
import numpy as np

import pylab as P

from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Read training data from file
df = pd.read_csv('./datasets/train.csv', header=0)

df.info()
df.describe()

In [ ]:
df[df['Age'].isnull()][['Sex', 'Pclass', 'Age']]

# Print number of men in eash class whose age is not present
for i in range(1,4):
    print i, len(df[ (df['Sex'] != 'female') & (df['Pclass'] == i) ])

In [ ]:
# Show a histogram for all non-null ages
df['Age'].dropna().hist(bins=16, range=(0,80), alpha = .5)
P.show()

In [ ]:
# Create gender numerous column
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df.head(3)

In [ ]:
# Create station numerous column (NA values are set to 0 as it's the most likely)
df[ df['Embarked'].isnull() ]

df['Station'] = df['Embarked'].dropna().map( {'S': 0, 'C': 1, 'Q': 2} )
df['Station'].dropna().hist()
P.show()

df['Station'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).fillna(0).astype(int)
df.head(3)

In [ ]:
# Create new age column and fill all null values there by median age basing on sex and class
median_ages = np.zeros((2,3))

for i in range(0,2):
    for j in range(0,3):
        median_ages[i,j] = df[ (df['Gender'] == i) & (df['Pclass'] == j+1) ]['Age'].dropna().median() 
        
median_ages

df['AgeFill'] = df['Age']
df.head()

for i in range(0,2):
    for j in range(0,3):
        df.loc[ (df['Age'].isnull()) & (df['Gender'] == i) & (df['Pclass'] == j+1), 'AgeFill' ] = median_ages[i,j]
        
df['AgeIsNull'] = pd.isnull(df['Age']).astype(int)
        
df[ df['Age'].isnull() ][['Gender', 'Pclass', 'Age', 'AgeFill', 'AgeIsNull']].head(10)

In [ ]:
df.describe()

In [ ]:
# Some feature engineering
df['FamilySize'] = df['SibSp'] + df['Parch']

df['Age*Class'] = df['AgeFill'] * df['Pclass']

df['FamilySize'].hist()
P.show()
df['Age*Class'].hist(bins=16, range=(0,250), alpha = .5)
P.show()

In [ ]:
# Drop unnecessary columns
df.dtypes[df.dtypes.map( lambda x: x == 'object' )]

df = df.drop(df[['PassengerId', 'Name', 'Sex', 'Age', 'Ticket', 'Cabin', 'Embarked']], axis=1)

df.info()

In [ ]:
# Convert data to an array
train_data = df.values
train_data

In [ ]:
forest = RandomForestClassifier(n_estimators=100)

# Train model
forest = forest.fit(train_data[0::,1::], train_data[0::,0])

In [ ]:
# Load test_data
df_test = pd.read_csv('./datasets/test.csv')
df_test.info()

In [ ]:
# Create gender numerous column
df_test['Gender'] = df_test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# Create station numerous column (NA values are set to 0)
df_test['Station'] = df_test['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# Fill age with median values
df_test['AgeFill'] = df_test['Age']

for i in range(0,2):
    for j in range(0,3):
        df_test.loc[ (df_test['Age'].isnull()) & (df_test['Gender'] == i) & (df_test['Pclass'] == j+1), 'AgeFill' ] = median_ages[i,j]
        
df_test['AgeIsNull'] = pd.isnull(df_test['Age']).astype(int)

# Fill fare with average value according to class
mean_fare = np.zeros((1,3))
for i in range(0,3):
    mean_fare[0,i] = df[ df['Pclass'] == i+1 ]['Fare'].mean()

for j in range(0,3):
    df_test.loc[ (df_test['Fare'].isnull()) & (df_test['Pclass'] == j+1), 'Fare' ] = mean_fare[0, j]
        
df_test[ df_test['Fare'].isnull() ]

In [ ]:
# Some feature engineering
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch']
df_test['Age*Class'] = df_test['AgeFill'] * df_test['Pclass']

In [ ]:
# Drop unnecessary columns
df_test_clean = df_test.drop(df_test[['PassengerId', 'Name', 'Sex', 'Age', 'Ticket', 'Cabin', 'Embarked']], axis=1)
df_test_clean.head(3)

In [ ]:
# Convert data to an array
test_data = df_test_clean.values
test_data

In [ ]:
# Predict results
output = forest.predict(test_data)

# Save to the file
predictions_file = open('./results/randomforestmodel.csv', 'wb')
p = csv.writer(predictions_file)
p.writerow(['PassengerId','Survived'])

for i in xrange(len(output)):
    p.writerow([df_test.loc[i,'PassengerId'], "%d" % output[i]])

predictions_file.close()